In [1]:
from smc import SMCController##引用库

In [2]:
##一次性连接并使能一个轴
smc = SMCController()
# 初始化连接
result = smc.board_init(0, 1, "COM11", 115200)
# 执行其他操作
smc.write_sevon_pin(0, 0,0)
#smc.basic_run(0)
# 关闭连接
#smc.board_close(0)

0

In [ ]:
####展示smc库目前已支持的函数
functions=smc.smc_dll.__dict__.keys()
print(len(functions))
for function in functions:
    print(function)


18
_name
_FuncPtr
_handle
smc_board_init
smc_board_close
smc_get_position_unit
smc_write_sevon_pin
smc_basic_run
smc_basic_stop
smc_set_home_profile_unit
smc_set_home_position_unit
smc_set_homemode
smc_get_homemode
smc_home_move
smc_ptt_table_unit
smc_pts_table_unit
smc_pvt_move
smc_t_pmove_extern_unit


In [8]:
#串口连接
result = smc.board_init(0, 1, "COM11", 115200)

In [12]:
#使能0轴
smc.write_sevon_pin(0, 0, 0)

0

In [21]:
#关闭0轴
smc.write_sevon_pin(0, 0, 1) 

0

In [17]:
# 按指定的方向和速度方式开始回原点
smc.set_home_profile_unit(0,0,99,90000,1,1)
smc.set_home_position_unit(0,0,1,0)
smc.set_homemode(0,0,0,1,0,1)
smc.home_move(0,0)
smc.get_homemode(0,0)

(0, 0, 3.0, 0, 1)

In [13]:
#设置ptt运动参数
ptime=[]
ppos=[]
for i in range(20):
    ptime.append(i+1)
    ppos.append(50000*(i+1))
#ptime=[1,2,3,4,5,6,7,8,9,10,11,12,13,]
#ppos=[10000,20000,30000]
smc.ptt_table_unit(0,0,10,ptime,ppos)

0

In [18]:
#设置pts运动参数
ptime=[0,1,2,3,4,5,6]
ppos=[0,15000,55000,140000,380000,650000,680000]
pper=[0,20,0,60,0,20,80]

smc.pts_table_unit(0,0,7,ptime,ppos,pper)

0

In [20]:
#开始PVT运动
smc.pvt_move(0,1,[0])   

0

In [29]:
#读取指令计数器值
smc.get_position_unit(0,0)

(0, 1000000.0)

In [ ]:
#运行ram中的basic程序
smc.basic_run(0)

In [ ]:
#停止运行basic程序
smc.basic_stop(0)

In [22]:
#断开0号控制器的连接
smc.board_close(0)

0